In [1]:
import spacy

In [2]:
nlp = spacy.load('fr_core_news_sm')

c:\Users\ocean\Documents\Projects\T-AIA-901-MPL_7\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sentence_id = "1"

In [4]:
sentences = [
    "I love flowers",
    # "Je vais à Marseille pour aller à Rome.",
    # "Je vais à Paris puis à Rome pour arriver à Milan",
    "Notre voyage débutera à New York et nous terminerons notre périple à Los Angeles, en passant par plusieurs villes emblématiques des États-Unis.",
    "Nous allons explorer l'Europe en partant de Madrid, en passant par Paris, Amsterdam et Berlin, pour enfin nous arrêter à Prague.",
    "L'aventure débutera à Tokyo, nous traverserons les montagnes jusqu'à Kyoto, puis nous achevons notre périple à Osaka.",
    "Nous prévoyons de visiter l'Italie en commençant par Rome, puis de nous rendre à Florence, pour finalement conclure notre voyage à Venise.",
    "Nous sommes partis de Milan pour finir à Montpellier",
    "Je vais de Paris puis à Rome.",
    "Nous sommes partis de Milan afin d'arriver à Montpellier" ,
    "Nous sommes partis de Milan" ,
    "Je vais partir de Paris et traverser la campagne française pour atteindre Nice." ,
    "Je vais partir de Pékin, explorer la Grande Muraille, puis continuer jusqu'à Xi'an avant de conclure mon voyage à Shanghai.",
    "Notre voyage débutera en Australie, à Sydney, puis nous longerons la côte est pour finalement arriver à Cairns.",
    "Je voudrais prendre le train de Marseille à Paris.",
    "Je souhaite voyager de Bastia à Paris pour aller à Lille",
    "On a pris l'avion d'Alger à Madrid pour finir à Montpellier"
]

In [5]:
def recognize(sentence):
    doc = nlp(sentence)
    for token in doc:
        print("token : ", token)
        print('\t', "token.dep_ : ", token.dep_) #Je ....
        print('\t', "token.pos_ : ", token.pos_) #Je ....
        for child in token.children:
                print('\t', '\t', "child : ", child)
                print('\t', '\t', "child.dep_ : ", child.dep_)
                print('\t', '\t', "child.pos_ : ", child.pos_)
                for subchild in child.children:
                    print('\t', '\t', '\t', "subchild : ", subchild)
                    print('\t', '\t', '\t', "subchild.dep : ", subchild.dep_)
                    print('\t', '\t', '\t', "subchild.pos_ : ", subchild.pos_)

https://universaldependencies.org/u/pos/index.html

In [6]:
def process_sentences(phrase):
    print(phrase) #L'aventure débutera à Tokyo, nous traverserons les montagnes jusqu'à Kyoto, puis nous achevons notre périple à Osaka.
    doc = nlp(phrase)
    departure = ""
    destination = ""

    for token in doc:
        ####################### VERBE SANS PP      
        if token.dep_ == 'ROOT' and token.pos_ == 'VERB': 
            for child in token.children:
                ######## DEPART                            
                if child.dep_ == 'obl:arg' and child.pos_ == 'PROPN' : # depart connecter au root
                    departure = child.text
                    for subchild in child.children:
                        if subchild.dep_ == 'flat:name' and subchild.pos_ == "PROPN":

                            departure += f" {subchild}"
                    print("departure 1 :", departure) #Kyoto

                if child.dep_ == 'advcl' and child.pos_ == 'VERB':
                    for subchild in child.children:
                        print(subchild)
                        if subchild.dep_ == 'obl:mod' and subchild.pos_ == 'PROPN':
                            departure = subchild.text
                        elif subchild.dep_ == 'obl:agent' and subchild.pos_ == 'PROPN':
                            departure = subchild.text
                    print("departure 2 :", departure) #Tokyo

                if child.dep_ == 'xcomp' and child.pos_ == 'VERB':
                    for subchild in child.children:
                        if subchild.dep_ == 'advcl' and subchild.pos_ == 'VERB':
                            for sub in subchild.children:
                                if sub.dep_ == 'obl:arg' and sub.pos_ == 'PROPN':
                                    departure = sub.text
                    print("departure 3 :", departure)

                
                ######## DEST
                if child.dep_ == 'obl:mod' and child.pos_ == 'PROPN':
                    destination = child.text
                if child.dep_ == 'advcl'and child.pos_ =='VERB': # arr qui est collee au verbe à l'infinitif
                    for subchild in child.children:
                        if subchild.dep_ == 'obl:arg' and subchild.pos_ == "PROPN":
                            destination = subchild.text
                if child.dep_ == 'conj' and child.pos_ == 'VERB':
                    for subchild in child.children:
                        if subchild.dep_ == 'obl:arg' and subchild.pos_ == "PROPN":
                            destination = subchild.text
                            for sub in subchild.children:
                                if sub.dep_ == 'flat:name' and sub.pos_ == "PROPN":
                                    destination += f" {sub}"
                if child.dep_ == 'xcomp' and child.pos_ == 'VERB':
                    for subchild in child.children:
                        if subchild.dep_ == 'obl:arg' and subchild.pos_ == 'PROPN':
                            destination = subchild.text
        ####################### VERBE AVEC PP
        if token.dep_ == 'ROOT' and token.pos_ == 'NOUN':
            for child in token.children:
                if child.dep_ == 'nmod' and child.pos_ == 'PROPN':
                    departure = child.text
                if child.dep_ == 'advcl'and child.pos_ =='VERB':
                    for subchild in child.children:
                        if subchild.dep_ == 'obl:arg' and subchild.pos_ == "PROPN":
                            destination = subchild.text

    print("Departure:", departure)
    print("Destination:", destination, '\n')
    return departure, destination

In [7]:
recognize(sentences[0])

token :  I
	 token.dep_ :  ROOT
	 token.pos_ :  NOUN
token :  love
	 token.dep_ :  advmod
	 token.pos_ :  ADV
token :  flowers
	 token.dep_ :  ROOT
	 token.pos_ :  ADJ
	 	 child :  love
	 	 child.dep_ :  advmod
	 	 child.pos_ :  ADV


In [8]:
departure, destination = process_sentences(sentences[0])

I love flowers
Departure: 
Destination:  



In [9]:
def annotate_trip(sentence_id):
    if departure and destination:
        return f"{sentence_id},{departure},{destination}"
    else:
        return f"{sentence_id},Code,NOT_TRIP"

In [10]:
def detect_language(sentence):
    doc = nlp(sentence)
    lang = doc.lang_
    print(lang)
    return lang


In [11]:

def annotate_language(sentence_id, sentence):
    lang = detect_language(sentence)
    if lang != 'fr':
        return f"{sentence_id},NOT_FRENCH,"
    else:
        return annotate_trip(sentence_id)


In [12]:
result = annotate_language(sentence_id, sentences[0])

fr


In [13]:
result

'1,Code,NOT_TRIP'